In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Utility Functions

This notebook contains evaluation and search helper functions that are used by multiple notebooks.

### Dependencies

In [ ]:
import chess

import import_ipynb
from Globals import Globals

#### chess.Board.is_winning
Finds whether a certain color is winning, i.e. if they are in an advantageous position (but may not necessarily win the game, they just have the advantage). This is the case if their total board score, i.e. piece values plus positional scores, is greater than that of the opponent. If both players have an equal score, they are both considered to be winning (and will therefore try to avoid draws; this should make the game more interesting).

###### __<u>Arguments</u>__
``color (chess.Color):``  
The color of which the advantageous or disadvantageous position should be determined.

###### __<u>Returns _(bool)_</u>__
Whether or not the given color is in an advantageous position (see definition above).

In [ ]:
def is_winning(self, color: chess.Color) -> bool:
    score = self.board_score(color)
    return (score >= 0)

chess.Board.is_winning = is_winning
del is_winning

<a id= 'BoardScore'></a>
#### chess.Board.board_score
Finds the total board score for a color, i.e. piece values plus positional scores. This score is positive if this color is in an advantageous position and negative if this color is in a disadvantageous position based on these two criteria.

Note that checkmate is not taken into account, as it is trivial to evaluate a board in this state and was therefore not considered worth the computation cost of seeing if a checkmate exists. This function is called to evaluate potential draws, therefore these computation costs could add up during a search. Furthermore, if a game has ended in a draw, a checkmate isn't possible, so this extra check would only be of use when showing the board score to the user.

###### __<u>Arguments</u>__
``color (chess.Color):``  
The color for which the board score should be calculated.

###### __<u>Returns _(int)_</u>__
The total board score for the given color.

In [ ]:
def board_score(self, color: chess.Color) -> int:
    score = 0
    for square in chess.SQUARES:
        piece = self.piece_at(square)
        if piece is None:
            continue
            
        piece_score_multiplier = 1 if piece.color == color else -1
        score += self.get_piece_value(piece.piece_type) * piece_score_multiplier
        piece_square_table_key = self.get_piece_square_table_key(piece.piece_type)
        score += self.get_positional_score(piece.color, piece_square_table_key, square) \
                * piece_score_multiplier

    return score

chess.Board.board_score = board_score
del board_score

#### chess.Board.get_piece_value
Finds the piece value for a given piece type. If the global constant contains a piece value for this piece type, it simply returns this piece value. If not, i.e. for kings or empty spaces, it returns a value of 0.

###### __<u>Arguments</u>__
``piece_type (chess.PieceType):``  
The piece type of which the piece value should be returned.

###### __<u>Returns _(int)_</u>__
The associated piece value, or 0 if not applicable.

In [ ]:
def get_piece_value(self, piece_type: chess.PieceType) -> int:
    piece_symbol = chess.piece_symbol(piece_type).lower()
    if piece_symbol in Globals.PIECE_VALUES:
        return Globals.PIECE_VALUES[piece_symbol]
    return 0

chess.Board.get_piece_value = get_piece_value
del get_piece_value

#### chess.Board.get_piece_square_table_key
Finds the dictionary key to index the piece square table of a piece type, which contains a positional score for each square on the chess board. Normally this key is simply the piece symbol of the piece type, but the king has a different piece square table key during the endgame.

###### __<u>Arguments</u>__
``piece_type (chess.PieceType):``  
The piece type to get the piece square table key for.

###### __<u>Returns _(str)_</u>__
The piece square table key associated with the piece type.

In [ ]:
def get_piece_square_table_key(self, piece_type: chess.PieceType) -> str:
    piece_symbol = chess.piece_symbol(piece_type).lower()
    if piece_symbol == 'k':
        return 'ke' if self.ending else 'km'
    return piece_symbol

chess.Board.get_piece_square_table_key = get_piece_square_table_key
del get_piece_square_table_key

#### chess.Board.find_castling_rook_squares
Finds the "from" and "to" square of the rook's movement during castling. This is necessary because the move's notation only contains the king's movement, which indirectly defines the rook's movement as well. <br>
The rook can only be in one of the four corners of the board prior to castling. Each corner also has a specified direction that the king moves in (1 if right and -1 if left). This function can therefore simply check for these four cases and place the rook behind the king in the correct direction.

###### __<u>Arguments</u>__
``color (chess.Color):``  
The color of the player who makes the move.

``move (chess.Move):``  
The castling move, describing the king's movement.

###### __<u>Returns _(chess.Square, chess.Square)_</u>__
- The "from" square of the rook's movement.
- The "to" square of the rook's movement.

In [ ]:
def find_castling_rook_squares(
    self, 
    color: chess.Color, 
    move: chess.Move
) -> (chess.Square, chess.Square):

    king_move_dir = 1 if move.to_square > move.from_square else -1
    old_rook_square = (chess.A1 if king_move_dir == -1 else chess.H1) \
        if color == chess.WHITE \
        else (chess.A8 if king_move_dir == -1 else chess.H8)
    new_rook_square = move.to_square - king_move_dir

    return old_rook_square, new_rook_square

chess.Board.find_castling_rook_squares = find_castling_rook_squares
del find_castling_rook_squares

#### chess.Board.get_state_repr
Creates a tuple that represents the current state of a chess board. En passant rights are not taken into account, as this requires an inefficient function call. Because en passant is relatively rare, this was not deemed worth the performance burden.

###### __<u>Returns _(tuple<int, int, int, int, int, int, int, int, int chess.Color>)_</u>__
A tuple that represents the board state. Each element in the tuple is a bitboard representation (`int`), except for the last element, which is the color that can currently make a move.

In [ ]:
def get_state_repr(self) -> tuple:
    return (
        self.pawns,
        self.knights,
        self.bishops,
        self.rooks,
        self.queens,
        self.kings,
        self.occupied_co[chess.WHITE],
        self.occupied_co[chess.BLACK],
        self.castling_rights,
        self.turn
    )
    

chess.Board.get_state_repr = get_state_repr
del get_state_repr

#### chess.Board.check_and_set_ending
Finds if the game has entered its ending phase. This follows the definition of the Simplified Evaluation Function, see [Michniewski, T. (2021)](Bibliography.ipynb#MT21). The ending phase is triggered when <b>either</b> of the following conditions is met:
- Both sides have no queens.
- Every side that has a queen has at most one minor piece and no other pieces (pawns are not counted).

If the ending phase is triggered, the `ending` variable of the board is set to `True`, so that the ending phase does not have to be checked for again. Depending on this variable, the kings will use different piece square tables.

In [ ]:
def check_and_set_ending(self) -> None:
    if self.ending: return

    meets_ending_requirement = { chess.WHITE: False, chess.BLACK: False }
    for color in [ chess.WHITE, chess.BLACK ]:
        # Option 1: no queen
        if self.get_piece_count(color, chess.QUEEN) == 0:
            meets_ending_requirement[color] = True
            continue

        # Option 2: at most one minor piece and no other pieces (besides queen)
        # Most sources state that pawns aren't pieces, so these are not counted
        if self.get_piece_count(color, chess.BISHOP, chess.KNIGHT) <= 1 \
                and self.get_piece_count(color, chess.ROOK) == 0:
            meets_ending_requirement[color] = True

    if meets_ending_requirement[chess.WHITE] and meets_ending_requirement[chess.BLACK]:
        self.ending = True

chess.Board.check_and_set_ending = check_and_set_ending
del check_and_set_ending

#### chess.Board.get_piece_count
Finds how many pieces of certain types of a certain color are on the board. For example, this function can be used to find the combined total of white bishops and knights.

###### __<u>Arguments</u>__
``color (chess.Color):``  
The color of the pieces that should be counted.

``*piece_types (chess.PieceType):``  
The piece type(s) that should be counted.

###### __<u>Returns _(int)_</u>__
The total amount of pieces of the given types and of the given color.

In [ ]:
def get_piece_count(self, color: chess.Color, *piece_types: chess.PieceType) -> int:
    return sum( len(self.pieces(piece_type, color)) for piece_type in piece_types )

chess.Board.get_piece_count = get_piece_count
del get_piece_count

#### chess.Board.update_check_endgame_tablebase
Checks if the endgame tablebases could reasonably return any results. From the file names in the Gaviota endgame tablebases, it can be found that no endgame tablebase considers positions with more than 5 pieces (with pawns counting as pieces in this instance). Therefore, if there are at most 5 pieces on the board, the `check_endgame_tablebase` flag is set. Until this flag is set, the endgame tablebases are not called during evaluation.

In [ ]:
def update_check_endgame_tablebase(self) -> None:
    if self.check_endgame_tablebase: return
    self.check_endgame_tablebase = (self.get_total_piece_count() <= 5)

chess.Board.update_check_endgame_tablebase = update_check_endgame_tablebase
del update_check_endgame_tablebase

#### chess.Board.get_total_piece_count
Finds how many pieces (and pawns) are on the board in total.

###### __<u>Returns _(int)_</u>__
The total amount of pieces (and pawns) on the board.

In [ ]:
def get_total_piece_count(self) -> int:
    return len([ square for square in chess.SQUARES if self.piece_at(square) is not None ])

chess.Board.get_total_piece_count = get_total_piece_count
del get_total_piece_count

#### chess.Board.get_search_result_if_finished
Checks if a search is finished. This is the case if either the game is over with the board state at the current search node, or if the remaining search depth is 0, i.e. if no further moves should be evaluated. If the search is finished, it returns the resulting move score of this search.

###### __<u>Arguments</u>__
``color (chess.Color):``  
The color of the player that started the search.  

``depth (int):``  
The remaining depth of the search, i.e. how many moves the search should still look into the future.  

###### __<u>Returns _(int)_</u>__
The move score if the search has ended, or `None` otherwise.

In [ ]:
def get_search_result_if_finished(
    self, 
    color: chess.Color, 
    depth: int, 
    eval_score: int
) -> int:

    # If the game has finished, return a positive score if we won,
    # or a negative score if we lost.
    # If the game ended in a draw, return a negative score if we are
    # in an advantageous position or a positive score otherwise.
    outcome = self.outcome()
    if outcome is not None:
        max_score = Globals.EVALUATION_SCORE_CHECKMATE + depth
        min_score = -Globals.EVALUATION_SCORE_CHECKMATE - depth
        if outcome.winner is None: # Draw
            return -Globals.EVALUATION_SCORE_DRAW if self.is_winning(color) \
                    else Globals.EVALUATION_SCORE_DRAW
        if outcome.winner == color: # Win
            return max_score
        return min_score # Loss
        
    # If no more moves should be evaluated, return the current evaluation score.
    if depth == 0:
        return eval_score

    # The search has not ended, so no result.
    return None

chess.Board.get_search_result_if_finished = get_search_result_if_finished
del get_search_result_if_finished